In [1]:
import re
import json
import sqlparse

In [115]:
#pyfile_path="../scripts/process1.py"
pyfile_path="../notebooks/process2.ipynb"
with open(pyfile_path) as f:
    content=f.read()

In [117]:
content=re.sub("[\s]+","",content)
content=re.sub("\\\\\"","\"",content)
content

'{"cells":[{"cell_type":"code","execution_count":null,"metadata":{},"outputs":[],"source":["importpandasaspd\\n","\\n","df1=pd.read_csv("a.csv")"]},{"cell_type":"code","execution_count":null,"metadata":{},"outputs":[],"source":["df2=pd.read_csv(\\n",""b.csv",\\n","sep=",",\\n","header=None\\n",")\\n","\\n","df3=pd.merge(df1,df2,on="ID")\\n","\\n","df4=pd.read_csv("c.csv",delimiter=":")"]},{"cell_type":"code","execution_count":null,"metadata":{},"outputs":[],"source":["\\n","df3.to_csv("d.csv",index=False)"]},{"cell_type":"code","execution_count":null,"metadata":{},"outputs":[],"source":[]}],"metadata":{"kernelspec":{"display_name":"base","language":"python","name":"python3"},"language_info":{"name":"python","version":"3.9.12(main,Apr42022,05:22:27)[MSCv.191664bit(AMD64)]"},"orig_nbformat":4,"vscode":{"interpreter":{"hash":"ad2bdc8ecc057115af97d19610ffacc2b4e99fae6737bb82f5d7fb13d2f2c186"}}},"nbformat":4,"nbformat_minor":2}'

In [118]:
json_path="../config/data_io_format.json"
with open(json_path) as f:
    io_format=json.load(f)

In [119]:
io_format

{'function_suffix': '\\("([^"]*)"',
 'input_pattern': ['pd.read_csv'],
 'output_pattern': ['to_csv']}

In [120]:
inp=io_format["input_pattern"][0]
suf=io_format["function_suffix"]

In [121]:
re.findall(inp+suf,content)

['a.csv', 'c.csv']